In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from os import path


from deep4deep.text_retrieval import prepare_my_df, get_meta_description_columns,get_meta_description,get_dealroom_meta_description
from deep4deep.text_processing import text_preprocessing, remove_special_chars, remove_punctuation
from deep4deep.w2v_embedding_and_rnn_model import change_to_categorical, Preprocessor, Embedder, LstmModel
from deep4deep import utils_w2v_rnn
from deep4deep.utils import simple_time_tracker

from deep4deep.trainer import LSTM_Meta_Trainer



from deep4deep import trainer
#comment éviter de recharger le glove à chaque fois? 
trainer.demo()

In [28]:
trainer = LSTM_Meta_Trainer()

initializing Trainer
data_prep 0.02
initializing Preprocessor
downloading transfer learning embedder – this may take time, coffee break maybe?
initializing Embedder
initializing LstmModel
init_and_compile_model 0.03


In [18]:
my_path = "../raw_data/data2020-12-03.csv"
recu_train, recu_test = train_test_split(pd.read_csv(my_path), test_size = .25)
recu_train.shape, recu_test.shape

((1134, 94), (379, 94))

In [22]:
train, val = train_test_split(recu_train, test_size = .25)
X_train_main_module = train.drop(columns=['target'])
y_train=train['target']
X_val_main_module = val.drop(columns=['target'])
y_val=val['target']

X_train_main_module.shape, X_val_main_module.shape

((850, 93), (284, 93))

In [30]:
# now I want it to return…

def merge_df (df_from_main_module, my_df_meta_categorical):
    # both have id as col (not index)
    df_from_main_module = df_from_main_module[['id']] # keep no other col
    # inner -> keep only the rows that are in both (i.e. selected in df_from_main_module,
    # and with data in )
    
    df = df_from_main_module.merge(my_df_meta_categorical, on='id', how='left', )
    df_with_meta = 
    
    return df

In [35]:
#lstm_training: define my X
#train = 
tmp = merge_df(X_train_main_module, trainer.df_meta_categorical)
#val = merge_df(X_val_main_module, self.df_meta_categorical)

In [36]:
tmp.shape

(850, 9)

In [51]:
tmp.full_text.dropna(axis=0, how='any').shape

(739,)

In [52]:
tmp[tmp.full_text.isna()]

,id,name,tagline,url,website_url,target,dealroom_meta_description,meta_description,full_text
0,919637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1749120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,1833275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,1832860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,1817219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
831,1891276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
838,1464546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
839,188523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
845,1447945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
tmp.isna().sum(axis=0)

id                             0
name                         107
tagline                      120
url                          107
website_url                  107
target                       107
dealroom_meta_description    107
meta_description             111
full_text                    111
dtype: int64

In [ ]:
#lstm_training: define my X
#train = 
merge_df(X_train_main_module, self.df_meta_categorical)
#val = merge_df(X_val_main_module, self.df_meta_categorical)
